In [1]:
!pip install tensorflow
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

Sun Apr 23 07:56:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Part 1 - Building the CNN
#importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
import matplotlib.pyplot as plt
import h5py

#Input Image Size
sz=300

In [5]:
# Initialing the CNN
classifier = Sequential()

In [6]:
# Step 1 - Convolutio Layer 
classifier.add(Convolution2D(32, (3,3) , input_shape = (sz, sz,1), activation = 'relu', padding='same'))

In [7]:
#step 2 - Pooling
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [8]:
# Adding second convolution layer
classifier.add(Convolution2D(32, (3,3) , activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [9]:
#Adding 3rd Concolution Layer
classifier.add(Convolution2D(64, (3,3) , activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size =(2,2)))


In [10]:
#Step 3 - Flattening
classifier.add(Flatten())

In [11]:
#Step 4 - Full Connection
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(36, activation = 'softmax')) # 36 Units are the value of no of Classes

In [12]:
#Compiling The CNN
classifier.compile(
              optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [13]:
# Classifier Summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 300, 300, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 150, 150, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 37, 37, 64)       0

In [14]:
#Part 2 Fittting the CNN to the image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [15]:
#Dataset Store in Variable
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Data/train',
        target_size=(sz, sz),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Data/test',
        target_size=(sz, sz),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

Found 7605 images belonging to 36 classes.
Found 3330 images belonging to 36 classes.


In [16]:
#Model Initialize
epochs=25
model = classifier.fit_generator(
        training_set,
        steps_per_epoch=337,	#No of Image in Traing Set 7200/batch size(32)=225
        epochs=epochs, #Previous value is 25
        validation_data = test_set,
        validation_steps =78 	#No of Image in Test Set 1800/batch size(32)=56
      )

<ipython-input-16-0994f195a884>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model = classifier.fit_generator(


Epoch 1/25
 24/237 [==>...........................] - ETA: 53:43 - loss: 4.1975 - accuracy: 0.0286

KeyboardInterrupt: ignored

In [ ]:
#Saving the model
classifier.save('/Models/Trained_model.h5')

# model_json = classifier.to_json()
# with open("/content/drive/MyDrive/Data/model-bw1.json", "w") as json_file:
#     json_file.write(model_json)
print('Model Saved')

print(model.history.keys())

In [ ]:
#Accuracy
model.history['accuracy']